In [1]:
#####################################################################################
class Plots:
    def __init__(self, SM):
        #Use the shrinking core simulation class
        self.inst_sm = SM
        
        #create widgets to hold plots 1,2,3
        self.plot2_output = widgets.Output()
        self.plot3_output = widgets.Output()
        self.plot_output = widgets.Output()
        self.plot4_output = widgets.Output(layout={'width': '420px', 'height': '170px'})#, 'border':'2px solid black'})
       
        #initialise plot2
        #self.plot2()
 

    def smallfig(self):
        with self.plot4_output:
            plt.figure(facecolor='#ADD8E6')  # Set figure size to match widget size
            plt.plot([0, 1], [1, 1], color='#ADD8E6')
            plt.axis('off')  # Turn off axis for a cleaner appearance
            plt.show()
        
        
    #function to update the plot showing simulation radius vs time        
    def update_plot3(self, r,t,p):
        #clear previous display
        self.plot3_output.clear_output(wait=True)
        #on the cleared display plot ..
        output_notebook()
        p3 = figure(title= f'CORE RADIUS vs TIME (R={r} T={t} P={p})', width = 400, height = 400, x_axis_label = "time (s)", y_axis_label = 'Radius of Uranium core (um)')
        with self.plot3_output:
            #new SCM simulation using inputted values
            sim = self.inst_sm.simulate(r,t,p)
            x = sim[0]
            y = sim[1]
            p3.line(x, y, legend_label = 'SCM')
            p3.legend.location = "top_left"
            show(p3)  
            
    #plotting the opening scene of the animation
    #This needs work - needs to be associated with a play button
    #needs the simulation data
    #needs to show an animated gif of reaction over time
#     def plot2(self):
#         with self.plot2_output:
#             fig,ax = plt.subplots(figsize = (4,4))
#             ax.set(xlim=(-0.5,0.5), ylim=(-0.5,0.5))
#             ax.axis('equal')
#             centre_x = 0
#             centre_y = 0
#             ax.add_patch(plt.Circle((centre_x,centre_y),0.3, color='green'))
#             ax.add_patch(plt.Circle((centre_x,centre_y),0.08, color='blue'))
#             ax.annotate("Ro", xy=(centre_x, centre_y), xytext=(0.3+0.011, -0.01),arrowprops=dict(arrowstyle="]->", color='yellow'))
#             ax.annotate("Rc", xy=(centre_x, centre_y), xytext=(-0.02, 0.08+0.02),arrowprops=dict(arrowstyle="]->", color='yellow'))
#             plt.show()
            
    def plot2(self, r,t,p):
        self.plot2_output.clear_output(wait=True)
        sim = self.inst_sm.simulate(r,t,p)
        output_notebook()
        p2 = figure(title= f'PROGRESSION vs TIME (R={r} T={t} P={p})', width = 400, height = 400, x_axis_label = "time (s)", y_axis_label = "Reaction progression")
        with self.plot2_output:
            x = sim[0]
            y = (sim[2]-np.min(sim[2]))
            y= ((y/np.max(y))-1)*-1
            
            p2.line(x, y, legend_label = 'SCM')
            p2.legend.location = "top_left"
            show(p2)            
   
    #this plots the experimental data selected from the drop down menu
    def update_plot(self, change):
        if type(change)==str:
            selected_function = change
        else:
            selected_function = change['new']
        self.plot_output.clear_output(wait=True)
        data = pd.read_csv('stakebake_data/'+selected_function)
        output_notebook()
        p1 = figure(title= f'Plot of {selected_function}', width = 400, height = 400)
        with self.plot_output:
            x = data.x
            y=data[' y']

            
            p1.circle(x, y, legend_label = selected_function)
            p1.legend.location = "top_left"
            show(p1)
#####################################################################################
class FunctionPlotter:
    def __init__(self, BW, PL):
        #initialise the instance of the button widgets class
        self.inst_bw = BW
        
        #initialise an instance of the plotting class
        self.inst_pl = PL
        
        self.inst_pl.update_plot(self.inst_bw.dropdown.value)
        
        
        simbox = widgets.VBox([self.inst_bw.radiobuttons, self.inst_bw.simulate])
        parambox = widgets.VBox([self.inst_bw.output_textboxes])
        box1 = widgets.HBox([simbox,parambox])
        topleft = widgets.VBox([widgets.HBox([self.inst_bw.upload, self.inst_bw.dropdown]),box1, 
                                self.inst_pl.plot4_output])
        topright = widgets.VBox([self.inst_pl.plot_output])
        
        
        
        toprow = widgets.HBox([topleft, topright])
        secondrow = widgets.HBox([self.inst_pl.plot2_output, self.inst_pl.plot3_output])
        thirdrow = widgets.HBox([self.inst_bw.fit_output, self.inst_bw.slider_output])

        display_layout = widgets.VBox([toprow, secondrow, thirdrow])


        custom_style = '<style>.widget-container { background-color: #ADD8E6; }</style>'
        display(HTML(custom_style))
        display(display_layout)

#####################################################################################
class ButtWidg:
    def __init__(self, SM, PL):
        
        #initialise an instance of plotting and shrinking core class
        self.inst_sm = SM
        self.inst_pl = PL
        
        #initialise all widgets
        self.upload_button()
        self.dropdown_button()
        self.radiobuttons()
        self.seemore()
        self.simulate()
        #self.de_slider()
        self.slider_output = widgets.Output()
        self.fit_output = widgets.Output()
        

        # Create an Output widget to hold the textboxes
        self.output_textboxes = widgets.Output()

        
        self.radiobuttons.observe(self.display_textboxes, names='value')


    #This is the dropdown button for experimental data selection        
    def dropdown_button(self):
        # find the current working directory
        self.current_directory = os.getcwd()
        #define a button widget where the options are the data files
        self.dropdown = widgets.Dropdown(
            options=self.get_file_list(self.current_directory+'/stakebake_data'),
            value = 'BM90.csv')
        # Observe the widgets to track changes
        self.dropdown.observe(self.inst_pl.update_plot, names='value')
    
    #This function is used to get the file names from data directory
    def get_file_list(self, folder_path):
        files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
        return files
        

    #This button displays every parameter for the shrinking core sim.
    def seemore(self):
        self.seemore = widgets.Button(description='See all parameters')
        # Set the event handler for the button click
        self.seemore.on_click(self.toggle_plot_text_boxes)

   # Function which displays more parameters if seemore clicked and gets back to the plot
    def toggle_plot_text_boxes(self, b):
        with self.inst_pl.plot4_output:
            clear_output(wait=True)
            if self.seemore.description == 'Hide all parameters':
                self.inst_pl.smallfig()
                self.seemore.description = 'See all parameters'
            else:
                self.tb1 = widgets.Text(value = '', description='Density U', layout=widgets.Layout(width='150px'))
                self.tb2 = widgets.Text(value = '', description='Density UH3',layout=widgets.Layout(width='150px'))
                self.tb3 = widgets.Text(value = '', description='b', layout=widgets.Layout(width='150px'))
                self.tb4 = widgets.Text(value = '', description='velocity', layout=widgets.Layout(width='150px'))
                self.tb5 = widgets.Text(value = '', description='density H', layout=widgets.Layout(width='150px'))
                self.tb6 = widgets.Text(value = '', description='a', layout=widgets.Layout(width='150px'))
                
                display(self.tb1, self.tb2,self.tb3,self.tb4,self.tb5,self.tb6)
                self.seemore.description = 'Hide all parameters'     

    #Button which runs simulation            
    def simulate(self):
        self.simulate = widgets.Button(description='Simulate')
        self.simulate.on_click(self.on_simulate_click)
    
    #Fetches the users values from textboxes and passes them to plot3 where sim is run
    def on_simulate_click(self,b):
        r = self.textbox1.value
        t = self.textbox2.value
        p = self.textbox3.value
        self.inst_pl.update_plot3(r,t,p)
        self.inst_pl.plot2(r,t,p)
        self.de_slider()
        self.fit()
        
        
    #This allows for a selection between simulatory models
    def radiobuttons(self):
        self.radiobuttons = widgets.RadioButtons(
            options=['Shrinking Core', 'Stakebake', 'CK'],
            value=None,  # No default selection
            description='Select Model:',
            disabled=False)
    
    #Button to upload own data - needs work
    def upload_button(self):
        self.upload = FileUpload(accept='.csv', multiple=True)
    

    #Dispplays textboxes where simulation parameters can be inserted      
    def display_textboxes(self, change):
        self.output_textboxes.clear_output(wait=True)

        if self.radiobuttons.value:  
            # Create multiple textboxes
            self.textbox1 = widgets.Text(value='300', description='',layout=widgets.Layout(width='100px'))
            self.textbox2 = widgets.Text(value='90', description='',layout=widgets.Layout(width='100px'))
            self.textbox3 = widgets.Text(value='10000', description='',layout=widgets.Layout(width='100px'))
             # Create descriptions with longer text
            desc1 = widgets.Label('Initial radius (um)', layout=widgets.Layout(width='150px'))
            desc2 = widgets.Label('Temperature (K)', layout=widgets.Layout(width='150px'))
            desc3 = widgets.Label('Pressure (Pa)', layout=widgets.Layout(width='150px'))
            
            butt = self.seemore

            # Display the textboxes and descriptions
            with self.output_textboxes:
                display(widgets.HBox([desc1, self.textbox1]))
                display(widgets.HBox([desc2, self.textbox2]))
                display(widgets.HBox([desc3, self.textbox3]))
                display(widgets.HBox([butt]))

        else:
            # Uncheck the checkbox and clear the output when it's unchecked
            self.checkbox.value = False
            self.close_textboxes(None)
            
    def de_slider(self):
        self.slider_output.clear_output(wait=True)
        with self.slider_output:
            self.slider = widgets.FloatSlider(
                value=0.0,   # Initial value
                min=-10.0,    # Minimum value
                max=10.0,     # Maximum value
                step=0.1,     # Step size
                description='Diffusivity:',  # Label for the slider
                orientation='horizontal')  # Orientation of the slider (horizontal or vertical)
            display(self.slider)
            
    def fit(self):
        self.fit_output.clear_output(wait=True)
        with self.fit_output:
            self.fit_button = widgets.Button(description='fit to experimental plot') 
            
            display(self.fit_button)
        



###########################################################################
class ShrinkingCore:
    def simulate(self,r,t,p):
        R = int(r)
        T = int(t)
        P = int(p)
        N = 100
        rho_u = 79663.866
        rho_uh3 =  45435.68 #0.045/(100**3)
        b = 0.67
        velocity = 0.5
        rho_h = 0.081
        p = 100000 #1 bar
        a=1.11e-10
        initial_pellet_size = 360
        r = scm.simulation(R,T,P,100,1e-10,rho_h,rho_u,rho_uh3,velocity)
        xdat = r[0]
        ydat = r[1]
        zdat = r[2]
        return xdat, ydat, zdat

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from ipywidgets import FileUpload
import os
import pandas as pd
import shrinkingcoremodel as scm
import warnings
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
warnings.filterwarnings('ignore')
SM = ShrinkingCore()
PL=Plots(SM)
bw=ButtWidg(SM, PL)

        
# Create an instance of the class
function_plotter = FunctionPlotter(bw, PL)


C:\Users\Emily Wright\AppData\Roaming\Python\Python39\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...